In [2]:
import pandas as pd

## Lectura de datos

In [3]:
cruces = pd.read_csv('cruces.csv', sep=';', encoding='latin1')
display(cruces.head())

,Codigo de vía tratado,Literal completo del vial tratado,Clase de la via tratado,Particula de la via tratado,Nombre de la via tratado,Codigo de via que cruza o enlaza,Literal completo del vial que cruza,Clase de la via que cruza,Particula de la via que cruza,Nombre de la via que cruza,Coordenada X (Guia Urbana) cm (cruce),Coordenada Y (Guia Urbana) cm (cruce),Longitud en S R WGS84 (cruce),Latitud en S R WGS84 (cruce)
0,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,2792,CALLE DE CASTIELLO DE JACA ...,CALLE,DE,CASTIELLO DE JACA,44260746,448479753,3º40'43.38'' W,40º30'36.13'' N
1,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,1837,CALLE DE FEDERICO MOMPOU ...,CALLE,DE,FEDERICO MOMPOU,44260732,448479748,3º40'43.43'' W,40º30'36.13'' N
2,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,87100,CALLE DE MARÍA TUBAU ...,CALLE,DE,MARIA TUBAU,44290433,448494484,3º40'30.85'' W,40º30'40.98'' N
3,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,2793,PASEO DE TIERRA DE MELIDE ...,PASEO,DE,TIERRA DE MELIDE,44320997,448510385,3º40'17.9'' W,40º30'46.21'' N
4,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,2794,CALLE DE PORTOMARÍN ...,CALLE,DE,PORTOMARIN,44322285,448530840,3º40'17.42'' W,40º30'52.83'' N


In [11]:
direcciones = pd.read_csv('direcciones.csv', sep=';', encoding='latin1')
# display(direcciones.head())
# direcciones['Coordenada X (Guia Urbana) cm'].astype('int64')
# print(direcciones['Coordenada X (Guia Urbana) cm'].dtype)
# direcciones['Coordenada Y (Guia Urbana) cm'].dtype = 'int64'
# display(direcciones[direcciones['Coordenada X (Guia Urbana) cm'].str.contains('-')])

C:\Users\Alex MGG\AppData\Local\Temp\ipykernel_23736\1376782993.py:1: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  direcciones = pd.read_csv('direcciones.csv', sep=';', encoding='latin1')


## Limpieza de datos

In [12]:
# Quitamos los espacios innecesarios de los datasets
for df in [cruces, direcciones]:
    for column in df.columns:
        if df[column].dtype == type(object):
            df[column] = df[column].str.strip()

In [8]:
# Ponemos una coordenada común de las glorietas (media de las coordenadas de los cruces)

# Para empezar nos quedamos con un dataset de las glorietas (solo necesitamos la glorieta como primera via)
cruces_glorietas = cruces[cruces['Clase de la via tratado'] == 'GLORIETA']

# Guardamos el número de cruces de cada glorieta en un diccionario, usando value_counts y diferenciando por el código de la vía
num_cruces = {}
for glorieta in cruces_glorietas['Codigo de vía tratado'].unique():
    num_cruces[glorieta] = cruces_glorietas['Codigo de vía tratado'].value_counts()[glorieta]

# Guardamos la suma de las coordenadas (diferenciamos entre X e Y) de los cruces de cada glorieta en un diccionario
coords_glorietas = {}
for glorieta in cruces_glorietas['Codigo de vía tratado'].unique():
    coords_glorietas[glorieta] = []
    coords_glorietas[glorieta].append(cruces_glorietas[cruces_glorietas['Codigo de vía tratado'] == glorieta]['Coordenada X (Guia Urbana) cm (cruce)'].sum())
    coords_glorietas[glorieta].append(cruces_glorietas[cruces_glorietas['Codigo de vía tratado'] == glorieta]['Coordenada Y (Guia Urbana) cm (cruce)'].sum())

# Calculamos la media de las coordenadas de los cruces de cada glorieta
for glorieta in coords_glorietas:
    coords_glorietas[glorieta][0] = coords_glorietas[glorieta][0] // num_cruces[glorieta]
    coords_glorietas[glorieta][1] = coords_glorietas[glorieta][1] // num_cruces[glorieta]

# Finalmente, en el dataset de cruces, si hay una glorieta en un cruce, ponemos las coordenadas de la glorieta
for index, row in cruces.iterrows():
    if row['Clase de la via tratado'] == 'GLORIETA':
        cruces.iloc[index, cruces.columns.get_loc('Coordenada X (Guia Urbana) cm (cruce)')] = coords_glorietas[row['Codigo de vía tratado']][0]
        cruces.iloc[index, cruces.columns.get_loc('Coordenada Y (Guia Urbana) cm (cruce)')] = coords_glorietas[row['Codigo de vía tratado']][1]
    elif row['Clase de la via que cruza'] == 'GLORIETA':
        cruces.iloc[index, cruces.columns.get_loc('Coordenada X (Guia Urbana) cm (cruce)')] = coords_glorietas[row['Codigo de via que cruza o enlaza']][0]
        cruces.iloc[index, cruces.columns.get_loc('Coordenada Y (Guia Urbana) cm (cruce)')] = coords_glorietas[row['Codigo de via que cruza o enlaza']][1]


In [13]:
# Primero, en la columna de Literal de numeracion, reemplazamos "KM.", " ", "EN", "SA", "NUM" por ""
# for replacement in ['KM.', ' ', 'EN', 'SA', 'NUM']:
#     direcciones['Literal de numeracion'] = direcciones['Literal de numeracion'].str.replace(replacement, '')

# Creamos una columna en la que almacenamos la direccion de la forma <codigo de via>-<literal de numeracion (sin letras)>
direcciones['Direccion'] = direcciones['Codigo de via'].astype(str) + '-' + direcciones['Literal de numeracion'].str.replace('[a-zA-Z.]', '')

direcciones = direcciones.groupby('Direccion').mean().reset_index()
direcciones

C:\Users\Alex MGG\AppData\Local\Temp\ipykernel_23736\3622352511.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  direcciones['Direccion'] = direcciones['Codigo de via'].astype(str) + '-' + direcciones['Literal de numeracion'].str.replace('[a-zA-Z.]', '')


,Direccion,Codigo de numero,Codigo de via,Codigo de distrito,Codigo de barrio,Seccion censal,Codigo postal,Zona Servicio Estacionamiento Regulado,Categoria fiscal
0,1000-000004,31047208.0,1000.0,16.0,4.0,73.0,28033.0,0.0,4.0
1,1000-000006,31047209.0,1000.0,16.0,4.0,73.0,28033.0,0.0,4.0
2,1000-000018,31041794.0,1000.0,16.0,4.0,71.0,28033.0,0.0,5.0
3,1000-000019,31041795.0,1000.0,16.0,4.0,71.0,28033.0,0.0,5.0
4,1000-000020,31041796.0,1000.0,16.0,4.0,71.0,28033.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...
193750,999-000068,20062296.0,999.0,17.0,4.0,105.0,28041.0,0.0,7.0
193751,999-000070,20062299.0,999.0,17.0,4.0,105.0,28041.0,0.0,7.0
193752,999-000072,20062302.0,999.0,17.0,4.0,105.0,28041.0,0.0,7.0
193753,999-000074,20062305.0,999.0,17.0,4.0,105.0,28041.0,0.0,7.0


## Cosas q hacer:

- Calles con numeros y letras unificarlas en un numero (hacer media de coordenadas)
- Introducir cruces en dataset de direcciones en su lugar correspondiente
- Navegar por los nodos, guardando informacion de las calles que cruzan, coordenadas y nodos adyacentes.
- Crear un grafo con las calles y sus cruces


## Dudas

- ¿Se puede crear una nueva columna?
- ¿Qué hacemos con los valores erróneos?